In [2]:
import pandas as pd
import re
from pydub.utils import mediainfo
import os
import jsonlines

In [86]:
df1 = pd.read_parquet('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00000-of-00014.parquet')

In [87]:
df = df1[(df1['language'] == 'English') & (df1['turn'] == 1)][['conversation']]

In [88]:
df.loc[:, 'conversation'] = df['conversation'].apply(lambda x: x[0]["content"])

In [211]:
df.head()

,conversation
0,Hey there! Are you familiar with reality shift...
2,"Old age PT hx of DM, HTN, dyslipidemia His ECG..."
4,Hey there! Are you familiar with reality shift...
7,Hey there! Are you familiar with reality shift...
8,i wanna you to write me terms & conditions and...


In [89]:
df = df.rename(columns={'conversation': 'source_text'})

In [90]:
df['id'] = range(1, len(df) + 1)
cols = ['id'] + [col for col in df.columns if col != 'id']
df = df[cols]

In [214]:
df.head()

,id,source_text
0,1,Hey there! Are you familiar with reality shift...
2,2,"Old age PT hx of DM, HTN, dyslipidemia His ECG..."
4,3,Hey there! Are you familiar with reality shift...
7,4,Hey there! Are you familiar with reality shift...
8,5,i wanna you to write me terms & conditions and...


In [91]:
def filter_text(text):
    if len(text.split()) > 30: 
        return False
    if not text or not isinstance(text, str):
        return False
    if re.search(r'http[s]?://\S+', text) or re.search(r'\S+@\S+', text):
        return False
    if re.search(r'[\x00-\x1F\x7F]+', text):
        return False
    if re.search(r'[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]{2,}', text):
        return False
    if re.search(r'[\$:]', text):
        return False
    if re.search(r'([+\-*/^=()])|%', text):
        return False
    if re.search(r"(\bimport\b|\bdef\b|\bclass\b|\bfunction\b|\bwrite\b|\bscript\b|\bmarkdown\b|\blatex\b|\bpython\b|\bjavascript\b|\bjava\b|\bc\+\+\b|\bgo\b|\bruby\b|\bphp\b|\btypescript\b|\breturn\b|\bfor\b|\bif\b|\belse\b|\bwhile\b|\btry\b|\bcatch\b|\bthrow\b|\basync\b|\bawait\b|\blambda\b|\bparameter\b|\bargument\b)", text.lower()):
        return False
    if re.search(r"\bprograme\b|\bcode\b|\bdebug\b|\bbug\b|gpt\b", text.lower()):
        return False
    if re.search(r'\\[dwsDWSbBtfnrvaA\+\*\.\?\(\)\[\]\{\}\|\\]', text):
        return False
    if re.search(r'[\U0001F600-\U0001F64F\u2700-\u27BF\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0]+', text):
        return False
    if re.search(r'\d+(st|nd|rd|th)', text.lower()):
        return False
    return True

In [92]:
df_filtered = df[df['source_text'].apply(filter_text)]

In [4]:
def filter(text):
    if len(text.split()) < 10: 
        return False
    if re.search(r'\b(what|why|how|give|who|when|which)\b', text.lower()):
        return True
    return False

In [218]:
df_filtered = df_filtered[df_filtered['source_text'].apply(filter)]

In [219]:
df_filtered.loc[:, 'id'] = range(1, len(df_filtered) + 1)
df_filtered.set_index('id', inplace=True, drop=False)
len(df_filtered)

631

In [220]:
df_filtered.head()

,id,source_text
id,,
1,1,Give me free apps that create midi files or sh...
2,2,What are some rhetorical devices to convice so...
3,3,what are the best ways to use a gpt4 chatgpt i...
4,4,I have 10 years of experience in web software ...
5,5,what's the best way to switch scenes behind a ...


In [5]:
def process_df(file):
    df = pd.read_parquet(file)
    df = df[(df['language'] == 'English') & (df['turn'] == 1)][['conversation']]
    df.loc[:, 'conversation'] = df['conversation'].apply(lambda x: x[0]["content"])
    df = df.rename(columns={'conversation': 'source_text'})
    df['id'] = range(1, len(df) + 1)
    cols = ['id'] + [col for col in df.columns if col != 'id']
    df = df[cols]
    df_filtered = df[df['source_text'].apply(filter_text)]
    df_filtered = df_filtered[df_filtered['source_text'].apply(filter)]
    return df_filtered

In [6]:
df1 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00000-of-00014.parquet')
df2 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00001-of-00014.parquet')
df3 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00002-of-00014.parquet')
df4 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00003-of-00014.parquet')
df5 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00004-of-00014.parquet')
df6 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00005-of-00014.parquet')
df7 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00006-of-00014.parquet')
df8 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00007-of-00014.parquet')
df9 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00008-of-00014.parquet')
df10 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00009-of-00014.parquet')
df11 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00010-of-00014.parquet')
df12 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00011-of-00014.parquet')
df13 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00012-of-00014.parquet')
df14 = process_df('/data/ruiqi.yan/data/speaking/WildChat-1M/data/train-00013-of-00014.parquet')
df_final = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14], ignore_index=True)

In [7]:
df_final.loc[:, 'id'] = range(1, len(df_final) + 1)
df_final.set_index('id', inplace=True, drop=False)
len(df_final)

8152

In [8]:
df_final.to_json('/data/ruiqi.yan/data/speaking/WildChat-1M/test1.jsonl', orient='records', lines=True, force_ascii=False)

In [71]:
prompt = """
Please help me select the data where the 'source_text' field corresponds to questions that meet the following conditions: 
### 1. The question should be relatively easy.
### 2. The question should be suitable for conversational contexts.
### 3. The question should be open-ended.
### 4. The question must not be related to creation.
### 5. The question must not contain proper nouns or abbreviations.
### 6. Remove the questions which are similar to each other.
### 7. Remove overly specialized questions.
### 8. The question is suitable for speech synthesis using TTS (Text-to-Speech) technology.

### Please keep the data you selected in its origin format (jsonl format) and delete the others.

The data is as follows:

"""

In [48]:
def save(n):
    l = min(n * 100 - 99, len(df_final))
    r = min(n * 100, len(df_final))
    file_path = '/data/ruiqi.yan/data/speaking/WildChat-1M/tmp.jsonl'
    df_final.loc[l:r, :].to_json(file_path, orient='records', lines=True, force_ascii=False)
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in jsonlines.Reader(f):
            data.append(line)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(prompt)
    with jsonlines.open(file_path, mode='a') as writer:
        for item in data:
            writer.write(item)
    with open(file_path, 'a', encoding='utf-8') as file:
        file.write("\n### Please check again and confirm that the data you selected meets the criteria.\n### Please keep the data you selected in its origin format (jsonl format) and delete the others.")

In [68]:
save(82)

In [72]:
def save_new(n):
    l = min(n * 100 - 99, len(df_final))
    r = min(n * 100, len(df_final))
    file_path = '/data/ruiqi.yan/data/speaking/WildChat-1M/tmp.jsonl'
    data = []
    with open("/data/ruiqi.yan/data/speaking/WildChat-1M/test.jsonl", 'r', encoding='utf-8') as f:
        for i, line in enumerate(jsonlines.Reader(f)):
            if i + 1 >= l and i + 1 <= r:
                data.append(line)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(prompt)
    with jsonlines.open(file_path, mode='a') as writer:
        for item in data:
            writer.write(item)
    with open(file_path, 'a', encoding='utf-8') as file:
        file.write("\n### Please check again and confirm that the data you selected meets the criteria.\n### Please keep the data you selected in its origin format (jsonl format) and delete the others.")

In [85]:
save_new(13)